<a href="https://colab.research.google.com/github/Deelaw15/Movie-Recommendation-System/blob/main/Matrix_Factorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install surprise

In [18]:
!pip install 'numpy<2'

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split, cross_validate
import pandas as pd
import os

In [21]:
!git clone https://github.com/Deelaw15/Movie-Recommendation-System.git

Cloning into 'Movie-Recommendation-System'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 76 (delta 35), reused 40 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (76/76), 8.15 MiB | 9.27 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [22]:
os.chdir('/content/Movie-Recommendation-System/ml-latest-small')

In [23]:
!ls

'd:\Data\Movie-Recommendation-System\best_svd_model.pkl'   ratings.csv
 links.csv						   README.txt
 Movie-Recommendation-System				   tags.csv
 movies.csv


In [24]:
train = pd.read_csv("ratings.csv")
train.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [25]:
# Load your train data (from earlier step)
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)


In [26]:
# Split into train/test for evaluation
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

In [27]:
# Initialize SVD model
svd_model = SVD(n_factors=100, biased=True, random_state=42)

In [28]:
# Train model
svd_model.fit(trainset)

In [29]:
# Evaluate
from surprise import accuracy
predictions = svd_model.test(testset)
rmse = accuracy.rmse(predictions)
print(f"✅ SVD RMSE: {rmse:.4f}")

RMSE: 0.8807
✅ SVD RMSE: 0.8807


Fine tuning

In [30]:
from surprise.model_selection import GridSearchCV

In [31]:
param_grid = {
    'n_factors': [50, 100, 150],
    'n_epochs': [20, 40, 60],
    'lr_all': [0.002, 0.005, 0.01],
    'reg_all': [0.02, 0.1, 0.2]
}

grid = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)
grid.fit(data)

In [32]:
print("✅ Best RMSE:", grid.best_score['rmse'])
print("🏆 Best Parameters:", grid.best_params['rmse'])

✅ Best RMSE: 0.857007717451643
🏆 Best Parameters: {'n_factors': 150, 'n_epochs': 60, 'lr_all': 0.01, 'reg_all': 0.1}


In [33]:
best_svd = SVD(n_factors=150, n_epochs=60, lr_all=0.01, reg_all=0.1)
full_trainset = data.build_full_trainset()
best_svd.fit(full_trainset)

In [34]:
import joblib

joblib.dump(best_svd, r'd:\Data\Movie-Recommendation-System\best_svd_model.pkl')
print("✅ Best SVD model saved successfully!")

✅ Best SVD model saved successfully!


In [35]:
!git config --global user.email "lawalfaruq50@gmail.com"
!git config --global user.name "Deelaw15"

In [37]:
joblib.dump(best_svd, "/content/best_svd_model.pkl")

['/content/best_svd_model.pkl']

In [38]:
!cp /content/best_svd_model.pkl /content/Movie-Recommendation-System/